<a href="https://colab.research.google.com/github/shuanglovesdata/Mar21-vaccine-uptake/blob/sc/DataDive_2021_Vaccine_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 18.4MB/s 
     |████████████████████████████████| 3.0MB 48.6MB/s 
     |████████████████████████████████| 890kB 55.5MB/s 
     |████████████████████████████████| 1.2MB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c0e4f39bb796f80bfef87bc38a28eb3d183716b5647cdb7321a67e6cd527fd93
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
from transformers import pipeline
import pandas as pd

#classifier = pipeline("zero-shot-classification")

#classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

# Load in more Tweets

In [26]:
TweetsDF=pd.read_csv( 'twitter_data_cleaned.csv')
TweetsDF=TweetsDF.drop_duplicates(subset=['full_text'])
VaccTweetsDF=TweetsDF[TweetsDF['full_text'].str.lower().str.contains("vac")]
sampledSequences=VaccTweetsDF.full_text.tolist()

In [27]:
len(sampledSequences)

31454

In [ ]:
%%time

sequences=sampledSequences[1001:1180]

candidate_labels = ["want", "don't want"]
hypothesis_template = "{} to get the Covid vaccine."

outputs=classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)#, multi_class=True)

outputDF=pd.DataFrame(outputs)
ExportDF=outputDF.set_index('sequence').apply(pd.Series.explode).reset_index()

In [9]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli')
tokenizer = AutoTokenizer.from_pretrained('joeddav/xlm-roberta-large-xnli')

sequence='People have stated that they will never take the vaccine, either because they are anti-vax, conspiracy theorists, or feel that the current vaccines are rushed and possibly dangerous',

premise = sequence
label = ["positive"]
hypothesis = f'This example is {label}.'

# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt',
                     truncation_strategy='only_first')
logits = nli_model(x)[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,1]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1796: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


In [8]:
hypothesis

"This example is ['positive', 'negative']."

In [5]:
logits

tensor([[-0.5448,  0.1261,  0.2942]], grad_fn=<AddmmBackward>)

In [6]:
entail_contradiction_logits

tensor([[-0.5448,  0.2942]], grad_fn=<IndexBackward>)

In [7]:
probs

tensor([[0.3017, 0.6983]], grad_fn=<SoftmaxBackward>)

In [4]:
prob_label_is_true

tensor([0.6983], grad_fn=<SelectBackward>)

In [6]:
#ExportDF.to_csv('VaccTweets_Label_1001_1250.csv')

In [16]:
import glob

path = '/content/Labelled' # use your path
all_files = glob.glob( path+ "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

LabelledDF = pd.concat(li, axis=0, ignore_index=True)

In [19]:
LabelledDF.head()

,Unnamed: 0,sequence,labels,scores
0,0,Please reply to this tweet if you want to shar...,want,0.987453
1,1,Please reply to this tweet if you want to shar...,don't want,0.012547
2,2,Gov. Ron DeSantis giving his rich friends the ...,want,0.980986
3,3,Gov. Ron DeSantis giving his rich friends the ...,don't want,0.019014
4,4,TRIPLE WORRISOME—Data says the new variant tha...,don't want,0.527854


In [21]:
LabelledDF[(LabelledDF.labels=="don't want") & (LabelledDF.scores>0.5)].shape

(108, 4)

In [22]:
antVacc_Tweets=LabelledDF[(LabelledDF.labels=="don't want") & (LabelledDF.scores>0.5)]

In [31]:
antVacc_Tweets=antVacc_Tweets[['sequence','labels','scores']].merge(VaccTweetsDF,  left_on='sequence', right_on='full_text')

In [34]:
T_USER_DF=pd.read_csv('twitter_user.csv')

In [48]:
antVacc_Tweets=antVacc_Tweets.merge(T_USER_DF,  how='left', on='user_id')

In [ ]:
antVacc_Tweets.to_csv('antVacc_Tweets.csv', index=False)

In [8]:
antVacc_Tweets=pd.read_csv('antVacc_Tweets_full.csv')

In [9]:
strong_antVacc_Tweets=antVacc_Tweets[antVacc_Tweets.scores>0.8]

In [10]:
strong_antVacc_Tweets.shape

(52, 17)

In [11]:
context=strong_antVacc_Tweets.full_text[:].tolist()

In [19]:
# summarizer = pipeline("summarization")
# summarizer(context, max_length=20, min_length=5, do_sample=False)

[{'summary_text': ' Steve Scalise is using his position to elevate Republicans who have publicly urged Americans not to'},
 {'summary_text': ' He never said he was anti vaccine. Hes anti covid vax. At least'},
 {'summary_text': ' Guy outside vaccination centre telling everyone #COVID-19 was a #Hoax .'},
 {'summary_text': ' Anti-vax campaigners, and Covid 19 sceptics, can come out with'},
 {'summary_text': ' Many assume reticence is an outgrowth of the ‘anti-vax'},
 {'summary_text': ' We need at least 70% of the population to be immunized to reach herd immunity'},
 {'summary_text': ' The amount of anti-vax folks in the beginning of this pandemic and the'},
 {'summary_text': ' Been working all the way thru this pandemic, haven’t caught anything,'},
 {'summary_text': " @senrobportman: Lets be clear during a pandemic this Senator doesn't"},
 {'summary_text': " It takes 6yrs to work towards a safe Vaccine also we Africans don't like"}]

In [23]:
QA = pipeline("question-answering")

result = QA(question="how many reasons for not taking the vaccine?", context=context[0])
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


Answer: 'anti-vax, conspiracy theorists,', score: 0.2788, start: 82, end: 113


In [14]:
%%time

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", return_dict=True)

# context = ['People have stated that they will never take the vaccine, either because they are anti-vax, conspiracy theorists, or feel that the current vaccines are rushed and possibly dangerous',\
#            'Protesters demonstrate against vaccine coercion, green passports in Tel Aviv. Anti-vax campaigners, and Covid 19 sceptics, can come out with the most offensive, and ridiculous, analogies ',\
#            'Less than 1 week after urging Republicans to block $160B in vaccine funding, Minority Whip @SteveScalise is using his position to elevate Republicans who have publicly urged Americans not to take the COVID-19 vaccine. ',
#            'Community Lattice exists to advance community development by equipping people with the resources they need to create economically sustainable, socially equitable, and resilient places.']

question = "what are the reasons for not taking COVID vaccine?"

answers=[]

for text in context:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    answers.append(answer)

    #print(f"Question: {question}")
    print(f"Answer: {answer}")

Answer: publicly urged americans not to take the covid - 19 vaccine
Answer: integrity
Answer: 
Answer: protesters demonstrate against vaccine coercion , green passports in tel aviv . anti - vax campaigners , and covid 19 sceptics , can come out with the most offensive , and ridiculous , analogies
Answer: the reasons health care workers have declined are complicated
Answer: they are anti - vax , conspiracy theorists , or feel that the current vaccines are rushed and possibly dangerous
Answer: the amount of anti - vax folks in the beginning of this pandemic and the scarcity of covid - 19
Answer: 
Answer: 
Answer: africans don ’ t like your human / animal experimentations
Answer: they don ’ t want to help biden stop the pandemic
Answer: inaccuracies and misinformation
Answer: 2 florida women busted while dressed up as “ grannies ”
Answer: she doesnt feel well
Answer: 2 florida women busted while dressed up as “ grannies ”
Answer: a sore arm and a rash
Answer: 
Answer: they are labelled an

In [15]:
answers[0:20]

['publicly urged americans not to take the covid - 19 vaccine',
 'integrity',
 '',
 'protesters demonstrate against vaccine coercion , green passports in tel aviv . anti - vax campaigners , and covid 19 sceptics , can come out with the most offensive , and ridiculous , analogies',
 'the reasons health care workers have declined are complicated',
 'they are anti - vax , conspiracy theorists , or feel that the current vaccines are rushed and possibly dangerous',
 'the amount of anti - vax folks in the beginning of this pandemic and the scarcity of covid - 19',
 '',
 '',
 'africans don ’ t like your human / animal experimentations',
 'they don ’ t want to help biden stop the pandemic',
 'inaccuracies and misinformation',
 '2 florida women busted while dressed up as “ grannies ”',
 'she doesnt feel well',
 '2 florida women busted while dressed up as “ grannies ”',
 'a sore arm and a rash',
 '',
 'they are labelled anti vaxers',
 'a virus . . . ? ? a pandemic',
 'get sick on purpose with so

In [13]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 9.6 GB  | Proc size: 6.3 GB
GPU RAM Free: 12589MB | Used: 2520MB | Util  17% | Total 15109MB
